In [76]:

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.decomposition import PCA

In [77]:
df = pd.read_csv("gurgaon_properties_just_before_feature_selection.csv").drop(columns = ["store room", "floortype", "balcony", "study room", "pooja room"])

In [78]:
df = df.drop_duplicates()

In [79]:
df["agePossession"].value_counts()

agePossession
Relatively New        1685
Moderately Old         618
New Property           594
Old Property           319
Under Construction     275
Name: count, dtype: int64

In [80]:
df['agePossession'] = df['agePossession'].replace(
    {
        'Relatively New':'new',
        'Moderately Old':'old',
        'New Property' : 'new',
        'Old Property' : 'old',
        'Under Construction' : 'under construction'
    }
)


In [81]:
df.head()
# just for simplicity 

,property_type,sector,price,bedRoom,bathroom,agePossession,built_up_area,servant room,others,furnishing_type,luxury_score
0,flat,sector 70,5.00,6.0,7.0,new,5333.00,1.0,0.0,0.0,medium
1,flat,sector 37d,0.60,2.0,2.0,new,625.00,0.0,0.0,0.0,low
2,flat,sector 79,1.90,3.0,4.0,new,2389.00,1.0,0.0,0.0,low
3,flat,sector 59,6.75,4.0,4.0,new,2567.00,1.0,0.0,0.0,low
4,flat,sector 49,1.98,3.0,4.0,old,1750.75,1.0,0.0,1.0,medium


In [82]:
df['property_type'] = df['property_type'].replace({'flat':0,'house':1})

C:\Users\guriv\AppData\Local\Temp\ipykernel_20096\71934247.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['property_type'] = df['property_type'].replace({'flat':0,'house':1})


In [83]:
df['luxury_score'] = df['luxury_score'].replace({'low':0,'medium':1,'high':2})

C:\Users\guriv\AppData\Local\Temp\ipykernel_20096\75390008.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['luxury_score'] = df['luxury_score'].replace({'low':0,'medium':1,'high':2})


In [84]:
df

,property_type,sector,price,bedRoom,bathroom,agePossession,built_up_area,servant room,others,furnishing_type,luxury_score
0,0,sector 70,5.00,6.0,7.0,new,5333.00,1.0,0.0,0.0,1
1,0,sector 37d,0.60,2.0,2.0,new,625.00,0.0,0.0,0.0,0
2,0,sector 79,1.90,3.0,4.0,new,2389.00,1.0,0.0,0.0,0
3,0,sector 59,6.75,4.0,4.0,new,2567.00,1.0,0.0,0.0,0
4,0,sector 49,1.98,3.0,4.0,old,1750.75,1.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
3535,1,sector 55,3.55,4.0,4.0,old,1449.00,0.0,0.0,1.0,0
3536,0,sector 77,1.09,3.0,4.0,new,1833.00,1.0,1.0,1.0,2
3537,0,sector 36a,0.40,1.0,1.0,new,436.00,0.0,0.0,0.0,0
3538,0,sector 90,1.23,3.0,3.0,new,1578.00,0.0,1.0,0.0,0


In [85]:
new_df= pd.get_dummies(df, columns = ["sector", "agePossession"], drop_first = True)

In [86]:
new_df

,property_type,price,bedRoom,bathroom,built_up_area,servant room,others,furnishing_type,luxury_score,sector_gwal pahari,...,sector_sector 92,sector_sector 93,sector_sector 95,sector_sector 99,sector_sector 99a,sector_sector 9a,sector_sohna road,sector_sohna road road,agePossession_old,agePossession_under construction
0,0,5.00,6.0,7.0,5333.00,1.0,0.0,0.0,1,False,...,False,False,False,False,False,False,False,False,False,False
1,0,0.60,2.0,2.0,625.00,0.0,0.0,0.0,0,False,...,False,False,False,False,False,False,False,False,False,False
2,0,1.90,3.0,4.0,2389.00,1.0,0.0,0.0,0,False,...,False,False,False,False,False,False,False,False,False,False
3,0,6.75,4.0,4.0,2567.00,1.0,0.0,0.0,0,False,...,False,False,False,False,False,False,False,False,False,False
4,0,1.98,3.0,4.0,1750.75,1.0,0.0,1.0,1,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,1,3.55,4.0,4.0,1449.00,0.0,0.0,1.0,0,False,...,False,False,False,False,False,False,False,False,True,False
3536,0,1.09,3.0,4.0,1833.00,1.0,1.0,1.0,2,False,...,False,False,False,False,False,False,False,False,False,False
3537,0,0.40,1.0,1.0,436.00,0.0,0.0,0.0,0,False,...,False,False,False,False,False,False,False,False,False,False
3538,0,1.23,3.0,3.0,1578.00,0.0,1.0,0.0,0,False,...,False,False,False,False,False,False,False,False,False,False


In [87]:
X = new_df.drop(columns = ["price"])
y= df["price"]

In [88]:
y_trans = np.log1p(y)

In [89]:
y_trans

0       1.791759
1       0.470004
2       1.064711
3       2.047693
4       1.091923
          ...   
3535    1.515127
3536    0.737164
3537    0.336472
3538    0.802002
3539    1.348073
Name: price, Length: 3491, dtype: float64

In [93]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [94]:
kfold = KFold(n_splits = 10, shuffle =  True, random_state=42)
lg = LinearRegression()
scores = cross_val_score(lg , X_scaled , y_trans, cv = kfold, scoring="r2")

In [101]:
scores.mean(), scores.std()

(0.8413427715905893, 0.0445200824124476)

In [104]:
lr = LinearRegression()

In [105]:
lr.fit(X_scaled, y_trans)

LinearRegression()

In [107]:
lr.coef_

(123,)

In [123]:
coef_df = pd.DataFrame(lr.coef_.reshape(1, -1), columns=X.columns).stack().reset_index().drop(columns=["level_0"]).rename(columns={"level_1": "features", 0: "coef"})


In [124]:
coef_df

,features,coef
0,property_type,0.122555
1,bedRoom,0.058401
2,bathroom,0.065100
3,built_up_area,0.208798
4,servant room,0.048568
...,...,...
118,sector_sector 9a,-0.004946
119,sector_sohna road,-0.027262
120,sector_sohna road road,-0.009366
121,agePossession_old,-0.010192


In [ ]:
# Read Regression Analysis 